<br>

# <font color="#2ecc71">Smart Contract Assistant </font>

<br>



| Section | Content |
|---------|--------|
| **1** | Configuration & API Setup |
| **2** | Document Ingestion Pipeline (ETL) |
| **3** | RAG Chain Logic |
| **4** | Interactive UI & Assessment |

| Component | Model | Runs |
|-----------|-------|------|
| **Embedding** | `sentence-transformers/all-MiniLM-L6-v2` | Locally (free) |
| **LLM** | `meta-llama/Llama-3.1-8B-Instruct` | Cloud (HF Inference API, free tier) |

---

## <font color="#2ecc71">Section 1: Configuration & API Setup</font>


In [63]:
# ── 1.1: Verify packages ───────────────────────────────────────────────
import importlib

packages = [
    'dotenv', 'langchain', 'langchain_community', 'langchain_huggingface',
    'langchain_text_splitters', 'faiss', 'gradio', 'pypdf', 'docx',
    'docx2txt', 'rich', 'pydantic', 'numpy', 'matplotlib'
]
missing = []
for pkg in packages:
    try:
        importlib.import_module(pkg)
    except ImportError:
        missing.append(pkg)

if missing:
    print(f'\u274c Missing packages: {missing}')
else:
    print('\u2705 All packages are installed and ready!')


✅ All packages are installed and ready!


In [64]:
# ── 1.2: Rich console setup ────────────────────────────────────────────
from functools import partial
from rich.console import Console
from rich.style import Style

console = Console()
base_style = Style(color="#2ecc71", bold=True)
pprint = partial(console.print, style=base_style)

pprint('\u2705 Rich console initialised.')


✅ Rich console initialised.

In [65]:
# ── 1.3: FIXED LLM Configuration ───────────────────────────────────────

import os, sys
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint, ChatHuggingFace

load_dotenv(dotenv_path=os.path.join('..', '.env'))

embedding_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2'
)
pprint('✅ Embedding model loaded (all-MiniLM-L6-v2, 384-dim).')

repo_id = "meta-llama/Llama-3.1-8B-Instruct" 

llm_endpoint = HuggingFaceEndpoint(
    repo_id=repo_id,
    task='text-generation',
    max_new_tokens=1100,         
    do_sample=True,              
    temperature=0.2,             
    top_p=0.95,
    repetition_penalty=1.4,
    huggingfacehub_api_token=os.environ.get('HUGGINGFACEHUB_API_TOKEN')
)

llm = ChatHuggingFace(llm=llm_endpoint)
pprint(f'✅ LLM ready: {repo_id} - DETAILED MODE')
pprint(f'   max_new_tokens=1100, temp=0.2, sampling=True')


✅ Embedding model loaded (all-MiniLM-L6-v2, 384-dim).

✅ LLM ready: meta-llama/Llama-3.1-8B-Instruct - DETAILED MODE

   max_new_tokens=1100, temp=0.2, sampling=True

In [66]:
# ── 1.4: Embedding sanity check ───────────────────────────────────────
import numpy as np

queries = [
    "What is the termination clause?",
    "When does the contract expire?",
    "What is the penalty for breach of contract?",
]

q_embeddings = [embedding_model.embed_query(q) for q in queries]

from sklearn.metrics.pairwise import cosine_similarity
sim_matrix = cosine_similarity(np.array(q_embeddings))

pprint('Cosine similarity between sample queries:')
for i in range(len(queries)):
    for j in range(i+1, len(queries)):
        print(f'  "{queries[i][:40]}..." \u2194 "{queries[j][:40]}..."  \u2192  {sim_matrix[i][j]:.4f}')

pprint(f'\n\u2705 Embedding model produces {len(q_embeddings[0])}-dimensional vectors.')


Cosine similarity between sample queries:

  "What is the termination clause?..." ↔ "When does the contract expire?..."  →  0.5063
  "What is the termination clause?..." ↔ "What is the penalty for breach of contra..."  →  0.4608
  "When does the contract expire?..." ↔ "What is the penalty for breach of contra..."  →  0.4828


✅ Embedding model produces 384-dimensional vectors.

In [67]:
# ── 1.5: Simple LCEL chain test ────────────────────────────────────────
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

test_prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful legal assistant. Be concise.'),
    ('human', '{input}'),
])

test_chain = test_prompt | llm | StrOutputParser()

pprint('Testing LLM with a simple question...')
result = test_chain.invoke({'input': 'What is a smart contract in 2 sentences?'})
print(f'\n{result}')
pprint('\n\u2705 LCEL chain works correctly!')


Testing LLM with a simple question...


A smart contract is a self-executing program stored on a blockchain that automates the enforcement and execution of a specific agreement or set of rules between parties. Once the contract's conditions are met, the code is triggered, allowing for the transfer of assets, execution of actions, or other predetermined outcomes without the need for intermediaries.


✅ LCEL chain works correctly!

In [68]:
# ── 1.6: Utility Runnables ─────────────────────────────────────────────
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from functools import partial

def RPrint(preface=""):
    """Simple passthrough 'prints, then returns' chain."""
    def print_and_return(x, preface):
        if preface:
            print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))


def docs2str(docs, title="Document"):
    """Convert a list of LangChain Document objects into a context string."""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('source', title)
        if doc_name:
            out_str += f"[Source: {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str


pprint('\u2705 Utility functions defined: RPrint(), docs2str()')


✅ Utility functions defined: RPrint(), docs2str()

---

## <font color="#2ecc71">Section 2: Document Ingestion Pipeline (ETL)</font>

Extract → Transform → Load pipeline: parse documents, split into chunks, embed, and store in FAISS.


In [69]:
# ── 2.1: Imports & paths (Fresh Start) ─────────────────────────────────
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import os, glob, time, shutil
import numpy as np

DATA_DIR = os.path.join('..', 'data')
VECTOR_STORE_PATH = os.path.join('..', 'vector_store')

if os.path.exists(DATA_DIR):
    shutil.rmtree(DATA_DIR)
if os.path.exists(VECTOR_STORE_PATH):
    shutil.rmtree(VECTOR_STORE_PATH)

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(VECTOR_STORE_PATH, exist_ok=True)

pprint('🗑️ Fresh start: data/ and vector_store/ cleared.')
pprint(f'   Data directory:    {os.path.abspath(DATA_DIR)}')
pprint(f'   Vector store path: {os.path.abspath(VECTOR_STORE_PATH)}')

🗑️ Fresh start: data/ and vector_store/ cleared.

   Data directory:    c:\Users\Computec\Desktop\ai_assisstant_contract\Smart_Contract_Assistant\data

   Vector store path: c:\Users\Computec\Desktop\ai_assisstant_contract\Smart_Contract_Assistant\vector_store

In [70]:
# ── 2.2: Document loader helper ────────────────────────────────────────

def load_document(file_path: str) -> list:
    """
    Load a single document file and return LangChain Document objects.
    Supports: .pdf, .docx, .txt
    """
    ext = os.path.splitext(file_path)[1].lower()
    filename = os.path.basename(file_path)

    if ext == '.pdf':
        docs = PyPDFLoader(file_path).load()
    elif ext == '.docx':
        docs = Docx2txtLoader(file_path).load()
    elif ext == '.txt':
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        docs = [Document(page_content=text, metadata={'source': filename})]
    else:
        raise ValueError(f'Unsupported file type: {ext}')

    for doc in docs:
        doc.metadata['source'] = filename

    pprint(f'  Loaded {len(docs)} page(s) from "{filename}"')
    return docs

pprint('\u2705 load_document() defined.')


✅ load_document() defined.

In [71]:
# ── 2.3: Text Splitter Configuration ───────────────────────────────────

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", ", ", " ", ""],
    length_function=len,
)

sample_text = (
    "This Agreement shall commence on the Effective Date and shall continue "
    "for a period of twelve (12) months. Either party may terminate this "
    "Agreement by providing thirty (30) days written notice. In the event "
    "of a material breach, the non-breaching party may terminate immediately "
    "upon written notice.\n\n"
    "The Contractor shall perform the Services described in Exhibit A "
    "in a professional and workmanlike manner. All deliverables shall "
    "conform to the specifications set forth in Exhibit B."
)

sample_chunks = splitter.split_text(sample_text)
pprint(f'Sample text ({len(sample_text)} chars) \u2192 {len(sample_chunks)} chunk(s):')
for i, chunk in enumerate(sample_chunks):
    print(f'  Chunk {i+1} ({len(chunk)} chars): "{chunk[:80]}..."')


Sample text (485 chars) → 1 chunk(s):

  Chunk 1 (485 chars): "This Agreement shall commence on the Effective Date and shall continue for a per..."


In [72]:
# ── 2.4: Cumulative Ingestion Pipeline ────────────────────────────────
def ingest_file(file_path: str) -> str:
    """
    ETL pipeline: Loads, splits, and MERGES into the existing FAISS index.
    """
    start_time = time.time()
    docs = load_document(file_path) 
    chunks = splitter.split_documents(docs) 

    new_store = FAISS.from_documents(chunks, embedding_model)

    if os.path.exists(os.path.join(VECTOR_STORE_PATH, 'index.faiss')):
        existing_store = FAISS.load_local(
            VECTOR_STORE_PATH, 
            embedding_model, 
            allow_dangerous_deserialization=True
        )
        existing_store.merge_from(new_store) 
        existing_store.save_local(VECTOR_STORE_PATH)
    else:
        new_store.save_local(VECTOR_STORE_PATH)
    
    elapsed = time.time() - start_time
    msg = f'✅ Ingested {len(chunks)} chunks from "{os.path.basename(file_path)}"'
    pprint(msg)
    return msg

In [73]:
# ── 2.5: Batch ingest all files  
import shutil

def ingest_all_files(reset=True):
    """
    Scan the data directory and ingest any PDF/DOCX/TXT files found.
    
    Args:
        reset: If True, clears the vector store first to avoid duplicates.
    """
    global vectorstore, retriever, rag_chain

    patterns = ['*.pdf', '*.docx', '*.txt']
    files = []
    for pattern in patterns:
        files.extend(glob.glob(os.path.join(DATA_DIR, pattern)))

    if not files:
        print(f'\u26a0\ufe0f No documents found in {os.path.abspath(DATA_DIR)}/')
        print('  Place your PDF, DOCX, or TXT files there and re-run this cell.')
        return

    if reset and os.path.exists(VECTOR_STORE_PATH):
        shutil.rmtree(VECTOR_STORE_PATH)
        os.makedirs(VECTOR_STORE_PATH, exist_ok=True)
        pprint('🗑️ Old vector store cleared (fresh start).')

    pprint(f'\n📁 Found {len(files)} file(s) to ingest:')
    for f in files:
        size_kb = os.path.getsize(f) / 1024
        print(f'  \u2022 {os.path.basename(f)} ({size_kb:.1f} KB)')
    print()

    total_chunks = 0
    for i, f in enumerate(files, 1):
        pprint(f'[{i}/{len(files)}] Processing: {os.path.basename(f)}')
        ingest_file(f)

    if os.path.exists(os.path.join(VECTOR_STORE_PATH, 'index.faiss')):
        vectorstore = FAISS.load_local(
            VECTOR_STORE_PATH, embedding_model,
            allow_dangerous_deserialization=True
        )
        total_chunks = len(vectorstore.docstore._dict)
        pprint(f'\n✅ Batch ingestion complete! Total chunks in index: {total_chunks}')
    else:
        pprint('\n\u26a0\ufe0f No index created.')


ingest_all_files(reset=True)

⚠️ No documents found in c:\Users\Computec\Desktop\ai_assisstant_contract\Smart_Contract_Assistant\data/
  Place your PDF, DOCX, or TXT files there and re-run this cell.


In [74]:
# ── 2.5b: Reset Vector Store (run ONLY when you want a clean start) ────
def reset_vector_store():
    """Delete the existing FAISS index to start fresh."""
    import shutil
    if os.path.exists(VECTOR_STORE_PATH):
        shutil.rmtree(VECTOR_STORE_PATH)
        os.makedirs(VECTOR_STORE_PATH, exist_ok=True)
        pprint('🗑️ Vector store cleared. Re-ingest your documents.')
    else:
        pprint('ℹ️ No vector store found to clear.')

# Uncomment to reset:
reset_vector_store()

🗑️ Vector store cleared. Re-ingest your documents.

In [75]:
# ── 2.6: Inspect the FAISS index ───────────────────────────────────────

faiss_index_file = os.path.join(VECTOR_STORE_PATH, 'index.faiss')
if os.path.exists(faiss_index_file):
    vectorstore = FAISS.load_local(
        VECTOR_STORE_PATH,
        embedding_model,
        allow_dangerous_deserialization=True,
    )
    total_chunks = len(vectorstore.docstore._dict)
    pprint(f'\u2705 FAISS index loaded: {total_chunks} chunks indexed.')

    test_query = 'termination clause'
    results = vectorstore.similarity_search_with_score(test_query, k=3)
    pprint(f'\nTop-3 results for "{test_query}":')
    for doc, score in results:
        source = doc.metadata.get('source', 'unknown')
        page = doc.metadata.get('page', '?')
        print(f'  [{score:.4f}] ({source}, p.{page}) "{doc.page_content[:100]}..."')
else:
    print('\u26a0\ufe0f No FAISS index found yet. Ingest some documents first.')
    print(f'   Place files in {os.path.abspath(DATA_DIR)} and run ingest_all_files().')


⚠️ No FAISS index found yet. Ingest some documents first.
   Place files in c:\Users\Computec\Desktop\ai_assisstant_contract\Smart_Contract_Assistant\data and run ingest_all_files().


---

## <font color="#2ecc71">Section 3: Retrieval-Augmented Generation (RAG) Chain</font>

Build the RAG pipeline: retriever, QA chain, conversation memory, evaluation.


In [76]:
# ── 3.1: Imports ───────────────────────────────────────────────────────
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_community.document_transformers import LongContextReorder
from operator import itemgetter
import json

pprint('\u2705 Imports OK')


✅ Imports OK

In [77]:
# ── 3.2: Load FAISS and create retriever  ──────────────


if os.path.exists(faiss_index_file):
    # Normal case: File exists, load it
    vectorstore = FAISS.load_local(
        VECTOR_STORE_PATH,
        embedding_model,
        allow_dangerous_deserialization=True,
    )
    total_chunks = len(vectorstore.docstore._dict)
    pprint(f'✅ Vector store loaded: {total_chunks} chunks indexed.')
else:
    pprint('⚠️ No index found. Creating a new FAISS index...')
    
    dummy_doc = [Document(page_content="system_initialization", metadata={'source': 'system'})]
    
    vectorstore = FAISS.from_documents(
        documents=dummy_doc,
        embedding=embedding_model,
    )
    
    os.makedirs(VECTOR_STORE_PATH, exist_ok=True)
    vectorstore.save_local(VECTOR_STORE_PATH)
    pprint('✅ New initial index created and saved.')

# ══════════════════════════════════════════════════════════════════════
# 3.2:Retriever - More chunks for detailed answers
# ══════════════════════════════════════════════════════════════════════

# ✅ CHANGED: k=8 instead of k=5 for more context
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 8},  # ✅ INCREASED from 5
)

pprint('✅ Retriever configured: top-8 similarity search (was 5)')

⚠️ No index found. Creating a new FAISS index...

✅ New initial index created and saved.

✅ Retriever configured: top-8 similarity search (was 5)

In [78]:
# ══════════════════════════════════════════════════════════════════════
# 3.3:  RAG Prompt - Demands Detailed Answers
# ══════════════════════════════════════════════════════════════════════


SYSTEM_PROMPT = """You are a professional legal assistant specialized in contract analysis.

CRITICAL INSTRUCTIONS:
1. **Provide DETAILED, COMPREHENSIVE answers** - at least 200-300 words
2. **Use ALL relevant information** from the context
3. **Structure your answer** with numbered points or bullet points
4. **Include specific details**: amounts, dates, percentages, conditions
5. **Cite sources** at the end: [Source: filename, Page: X]
6. **Explain thoroughly** - never summarize, always explain in full detail
7. **If asked about terms/clauses, list EACH ONE separately with full explanation**
8. For Arabic questions, respond in Arabic with the same level of detail

If context is insufficient, say: "لا توجد معلومات كافية في الوثائق المرفوعة" (in Arabic) or "I don't have enough information" (in English).

---
RETRIEVED CONTEXT:
{context}
---"""

rag_prompt = ChatPromptTemplate.from_messages([
    ('system', SYSTEM_PROMPT),
    ('human', '{input}'),
])

pprint('✅ RAG prompt FIXED - Now demands detailed answers')

✅ RAG prompt FIXED - Now demands detailed answers

In [79]:
# ── 3.5: Advanced LCEL RAG Chain (FIXED) ──────────────────────────────
from collections import defaultdict

def format_context(docs):
    """Format retrieved documents, GROUPED by source file."""
    grouped = defaultdict(list)
    for doc in docs:
        source = doc.metadata.get('source', 'unknown')
        grouped[source].append(doc)

    parts = []
    for source, source_docs in grouped.items():
        parts.append(f'\n{"="*50}')
        parts.append(f'📄 DOCUMENT: {source}')
        parts.append(f'{"="*50}')
        for doc in source_docs:
            page = doc.metadata.get('page', '?')
            parts.append(f'[Page {page}]:\n{doc.page_content}')
    
    return '\n\n'.join(parts)


# ✅ Dynamic retriever
retrieval_chain = (
    {'input': lambda x: x['input'] if isinstance(x, dict) else x}
    | RunnableAssign({
        'context_docs': (
            itemgetter('input')
            | RunnableLambda(lambda q: retriever.invoke(q))
            | LongContextReorder().transform_documents
        )
    })
)

generation_chain = (
    RunnableAssign({
        'context': lambda d: format_context(d['context_docs'])
    })
    | rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain = retrieval_chain | RunnableAssign({'answer': generation_chain})

pprint('\u2705 RAG chain built (grouped context).')

✅ RAG chain built (grouped context).

In [80]:
# ── 3.6: ask_question helper ───────────────────────────────────────────

def ask_question(query: str):
    """
    Send a question through the RAG chain.
    Returns (answer, source_pages, context_docs).
    """
    response = rag_chain.invoke({'input': query})
    answer = response['answer']
    docs = response.get('context_docs', [])

    sources = []
    for doc in docs:
        source = doc.metadata.get('source', 'unknown')
        page = doc.metadata.get('page', '?')
        sources.append(f'{source} (p.{page})')
    unique_sources = list(dict.fromkeys(sources))

    return answer, unique_sources, docs


pprint('\u2705 ask_question() defined.')


✅ ask_question() defined.

In [81]:
# ── 3.7: Pydantic Knowledge Base ──────────────────────────────────────
from pydantic import BaseModel, Field
from typing import Dict, List, Optional


class ContractKnowledge(BaseModel):
    """Running knowledge base for the current contract analysis session."""
    document_name: str = Field('unknown', description='Name of the uploaded document')
    parties: List[str] = Field([], description='Parties involved in the contract')
    key_dates: Dict[str, str] = Field({}, description='Important dates')
    key_clauses: List[str] = Field([], description='Key clauses identified')
    financial_terms: Dict[str, str] = Field({}, description='Payment amounts, fees, penalties')
    summary: str = Field('No summary yet.', description='Running summary of findings')
    unresolved: List[str] = Field([], description='Questions that could not be answered')


session_knowledge = ContractKnowledge()
conversation_history = []

pprint('\u2705 ContractKnowledge model defined.')
pprint(f'   Fields: {list(ContractKnowledge.model_fields.keys())}')


✅ ContractKnowledge model defined.

   Fields: ['document_name', 'parties', 'key_dates', 'key_clauses', 'financial_terms', 'summary', 'unresolved']

In [82]:
# ── 3.8: Conversational ask with history ───────────────────────────────

def ask_with_history(query: str) -> str:
    """Ask a question with conversation history context."""
    global conversation_history

    answer, sources, docs = ask_question(query)

    conversation_history.append({'role': 'user', 'content': query})
    conversation_history.append({'role': 'assistant', 'content': answer})

    sources_str = ', '.join(sources) if sources else 'No sources'
    formatted = f'{answer}\n\n\U0001f4c4 Sources: {sources_str}'

    return formatted


pprint('\u2705 ask_with_history() defined.')


✅ ask_with_history() defined.

In [83]:
# ── 3.9: RAG Evaluation Chain (LLM-as-a-Judge) ────────────────────────

EVAL_PROMPT = """You are an expert evaluator for RAG systems.
Given a QUESTION, the RETRIEVED CONTEXT, and the GENERATED ANSWER,
evaluate the answer on three criteria (each scored 1-5):

1. **Relevance**: Does the answer address the question? (1=irrelevant, 5=perfectly relevant)
2. **Groundedness**: Is the answer supported by the context? (1=hallucinated, 5=fully grounded)
3. **Completeness**: Does it cover all aspects? (1=incomplete, 5=comprehensive)

Respond ONLY with valid JSON in this exact format:
{{
    "relevance": <1-5>,
    "groundedness": <1-5>,
    "completeness": <1-5>,
    "overall": <1-5>,
    "reasoning": "<brief explanation>"
}}

QUESTION: {question}

RETRIEVED CONTEXT:
{context}

GENERATED ANSWER:
{answer}"""

eval_prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a precise RAG evaluator. Output ONLY valid JSON.'),
    ('human', EVAL_PROMPT),
])

eval_chain = eval_prompt | llm | StrOutputParser()

pprint('✅ eval_prompt and eval_chain defined.')

✅ eval_prompt and eval_chain defined.

In [84]:
def evaluate_answer(question: str, answer: str, context_docs: list) -> dict:
    """Evaluate a RAG answer using LLM-as-a-Judge."""
    context_str = format_context(context_docs)
    
    clean_answer = clean_markdown(answer) if 'clean_markdown' in globals() else answer
    
    result = eval_chain.invoke({
        'question': question,
        'context': context_str,
        'answer': clean_answer,
    })

    try:
        clean = result.strip()
        if clean.startswith('```'):
            clean = clean.split('\n', 1)[1].rsplit('```', 1)[0]
        return json.loads(clean)
    except json.JSONDecodeError:
        return {'raw_response': result, 'error': 'Failed to parse JSON'}

pprint('✅ evaluate_answer() defined.')

✅ evaluate_answer() defined.

---

## <font color="#2ecc71">Section 4: Interactive UI & Assessment</font>

Gradio web interface with Upload, Chat, and Analysis tabs.


In [85]:
# ── 4.1: Imports & Configuration ──────────────────────────────────────
import gradio as gr
import shutil, time
from datetime import datetime

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(VECTOR_STORE_PATH, exist_ok=True)

session_stats = {
    'documents_uploaded': [],
    'total_chunks': 0,
    'questions_asked': 0,
    'session_start': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
}

pprint('\u2705 UI module imports OK')


✅ UI module imports OK

In [86]:
# ── 4.2: Automatic Multi-File Cumulative Upload handler ────────────────

def process_upload(files):
    """
    Handles multiple files automatically when uploaded.
    New files are merged into the existing FAISS index.
    """
    global vectorstore, retriever, retrieval_chain, rag_chain, session_stats

    if files is None or len(files) == 0:
        return '⚠️ No files detected.'

    try:
        start_time = time.time()
        new_filenames = []

        for file_obj in files:
            if hasattr(file_obj, 'name'):
                filename = os.path.basename(file_obj.name)
                src = file_obj.name
            else:
                filename = os.path.basename(file_obj)
                src = file_obj

            dest = os.path.join(DATA_DIR, filename)
            import shutil
            shutil.copy(src, dest)
            ingest_file(dest)
            new_filenames.append(filename)

        # ✅ Reload vectorstore
        vectorstore = FAISS.load_local(
            VECTOR_STORE_PATH, embedding_model,
            allow_dangerous_deserialization=True,
        )

        # ✅ Rebuild retriever
        retriever = vectorstore.as_retriever(
            search_type='similarity', search_kwargs={'k': 8},
        )

        # ✅ Rebuild ENTIRE chain (this is the fix!)
        retrieval_chain = (
            {'input': lambda x: x['input'] if isinstance(x, dict) else x}
            | RunnableAssign({
                'context_docs': (
                    itemgetter('input')
                    | retriever
                    | LongContextReorder().transform_documents
                )
            })
        )

        generation_chain = (
            RunnableAssign({'context': lambda d: format_context(d['context_docs'])})
            | rag_prompt | llm | StrOutputParser()
        )
        rag_chain = retrieval_chain | RunnableAssign({'answer': generation_chain})

        # Update stats
        for name in new_filenames:
            if name not in session_stats['documents_uploaded']:
                session_stats['documents_uploaded'].append(name)

        current_total_chunks = len(vectorstore.docstore._dict)
        session_stats['total_chunks'] = current_total_chunks

        return f'✅ Ingested {len(new_filenames)} file(s). Total chunks: {current_total_chunks}'

    except Exception as e:
        return f'❌ Ingestion Error: {str(e)}'

pprint('✅ Multi-file process_upload() defined.')

✅ Multi-file process_upload() defined.

In [87]:
# ── 4.3: Chat handler (With DeepSeek Formatting Fixes) ─────────────────
import re

def clean_markdown(text):
    """
    Helper to fix common markdown formatting errors and remove internal 
    Chain-of-Thought logs (<think> tags).
    """
    if not text: return ""
    
    
    cleaned = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    
   
    cleaned = re.sub(r'([^\n])\s*\n\*\s', r'\1\n\n* ', cleaned)
    
    cleaned = re.sub(r'([^\n])\s*(#{1,6}\s)', r'\1\n\n\2', cleaned)
    
    return cleaned.strip()

def chat_fn(message, history):
    """Gradio chat callback with RAG and source citations."""
    global session_stats

    if not message or not message.strip():
        return '⚠️ Please type a question about your uploaded document(s).'

    faiss_index_file = os.path.join(VECTOR_STORE_PATH, 'index.faiss')
    if not os.path.exists(faiss_index_file):
        return (
            '⚠️ No documents indexed yet!\n'
            'Please go to the "Upload Document" tab and upload a PDF or DOCX file first.'
        )

    try:
        answer, sources, docs = ask_question(message)
        session_stats['questions_asked'] += 1

        formatted_answer = clean_markdown(answer)

        if sources:
            sources_list = "\n".join([f"- {s}" for s in sources])
            response = f"{formatted_answer}\n\n---\n### 📄 Sources Used:\n{sources_list}"
        else:
            response = f"{formatted_answer}\n\n---\n*No specific sources cited.*"

        return response

    except Exception as e:
        return f'❌ Error processing question: {str(e)}'

pprint('✅ chat_fn() defined with <think> tag removal.')

✅ chat_fn() defined with <think> tag removal.

In [88]:
import os
import json

# --- 4.4: Dashboard functions ------------------------------------------

def get_session_stats():
    """Return formatted session statistics."""
    # Handle the case where no documents are uploaded yet
    docs_list = '\n'.join(f'  {d}' for d in session_stats.get('documents_uploaded', [])) or '  (none)'
    
    # Properly closed f-string block
    stats = (
        f"SESSION STATISTICS\n"
        f"Session started:  {session_stats.get('session_start', 'N/A')}\n"
        f"Documents uploaded:\n{docs_list}\n"
        f"Total chunks:     {session_stats.get('total_chunks', 0)}\n"
        f"Questions asked:  {session_stats.get('questions_asked', 0)}\n"
    )
    return stats


def run_sample_evaluation():
    """Run a quick evaluation on sample questions to test RAG performance."""
    faiss_index_file = os.path.join(VECTOR_STORE_PATH, 'index.faiss')
    
    if not os.path.exists(faiss_index_file):
        return 'Upload a document first to run evaluation.'

    try:
        sample_questions = [
            'What are the main terms of this contract?',
            'What is the termination clause?',
            'What are the payment terms?',
        ]

        results = []
        for q in sample_questions:
            answer, sources, docs = ask_question(q)
            eval_result = evaluate_answer(q, answer, docs)

            if 'error' not in eval_result:
                results.append({
                    'question': q,
                    'relevance': eval_result.get('relevance', '?'),
                    'groundedness': eval_result.get('groundedness', '?'),
                    'completeness': eval_result.get('completeness', '?'),
                    'overall': eval_result.get('overall', '?'),
                })
            else:
                results.append({'question': q, 'error': eval_result.get('error', 'Unknown')})

        report = 'RAG EVALUATION REPORT\n' + '=' * 50 + '\n\n'
        for r in results:
            report += f"Q: {r['question']}\n"
            if 'error' in r:
                report += f"  Error: {r['error']}\n"
            else:
                report += f"  Relevance:    {r['relevance']}/5\n"
                report += f"  Groundedness: {r['groundedness']}/5\n"
                report += f"  Completeness: {r['completeness']}/5\n"
                report += f"  Overall:      {r['overall']}/5\n"
            report += '\n'

        # Calculate average score for valid numerical results
        valid = [r for r in results if 'overall' in r and isinstance(r['overall'], (int, float))]
        if valid:
            avg = sum(r['overall'] for r in valid) / len(valid)
            report += f'{"-" * 50}\n'
            report += f'⭐ Average Overall Score: {avg:.1f}/5\n'

        return report

    except Exception as e:
        return f'Evaluation error: {str(e)}'


def export_knowledge():
    """Export the session knowledge base and history as a JSON string."""
    data = {
        'session_stats': session_stats,
        # model_dump() assumes session_knowledge is a Pydantic model
        'knowledge_base': session_knowledge.model_dump() if session_knowledge else {},
        'conversation_history': conversation_history,
    }
    return json.dumps(data, indent=2, default=str)


print('Dashboard functions defined.')

Dashboard functions defined.


In [89]:
# ── 4.4: UI Wrappers ──────────────────────────────────────────────────

def streaming_chat(message, history):
    """Adapts chat_fn for the Gradio streaming interface."""
    response = chat_fn(message, history) 
    yield response

def auto_ingest_wrapper(files):
    """Handles the 'change' trigger for automatic ingestion."""
    if files is None:
        return "Waiting for files..."
    return process_upload(files)

pprint("✅ Wrappers 'streaming_chat' and 'auto_ingest' are now defined.")

✅ Wrappers 'streaming_chat' and 'auto_ingest' are now defined.

In [90]:
# ── Reset Vector Store (Run ONCE to fix duplicate/wrong retrievals) ────


if os.path.exists(VECTOR_STORE_PATH):
    shutil.rmtree(VECTOR_STORE_PATH)
    os.makedirs(VECTOR_STORE_PATH, exist_ok=True)
    print('🗑️ Vector store cleared successfully!')
else:
    print('ℹ️ No vector store found.')

🗑️ Vector store cleared successfully!


In [91]:
# ── Re-ingest all documents cleanly ────────────────────────────────────
ingest_all_files()

⚠️ No documents found in c:\Users\Computec\Desktop\ai_assisstant_contract\Smart_Contract_Assistant\data/
  Place your PDF, DOCX, or TXT files there and re-run this cell.


In [92]:
import gradio as gr
import os
import time

# --- 4.5: Fixed UI: Multi-file selection + Automatic Ingestion -----------

custom_css = """
.gradio-container { max-width: 1200px !important; margin: auto !important; }
.main-header { text-align: center; margin-bottom: 20px; padding: 10px; border-bottom: 1px solid #e0e0e0; }
.upload-zone { background: #f8f9fa; border: 2px dashed #2ecc71 !important; border-radius: 10px !important; }
"""

with gr.Blocks(title='Smart Contract AI') as app:

    with gr.Group(elem_classes="main-header"):
        gr.Markdown("# 📜 Smart Contract Assistant")
        gr.Markdown("### Multi-file upload is now enabled. Ingestion starts automatically.")

    with gr.Tabs():
        # TAB 1: Chat & Auto-Upload
        with gr.Tab("💬 Chat & Upload"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### 📤 Document Upload")
                    file_input = gr.File(
                        label="Select multiple files (PDF/DOCX)",
                        file_types=['.pdf', '.docx', '.txt'],
                        file_count="multiple", # Enables multi-file selection
                        type='filepath',
                        elem_classes="upload-zone"
                    )
                    
                    status_box = gr.Textbox(
                        label="Ingestion Status",
                        placeholder="Automatic ingestion will start after selection...",
                        interactive=False,
                        lines=4
                    )
                    
                    # TRIGGER: Auto-ingest when files are selected/changed
                    file_input.change(
                        fn=auto_ingest_wrapper,
                        inputs=file_input,
                        outputs=status_box
                    )

                with gr.Column(scale=3):
                    chatbot = gr.ChatInterface(
                        fn=streaming_chat,
                        fill_height=True,
                        examples=["Summarize all uploaded contracts.", "What are the common payment terms?"]
                    )

        # TAB 2: Analysis Dashboard (Restored)
        with gr.Tab("📊 Analysis & Export"):
            gr.Markdown("### 🛠️ Session Analysis Tools")
            with gr.Row():
                stats_btn = gr.Button("📊 View Session Stats", variant="secondary")
                eval_btn = gr.Button("🧪 Run Quality Eval", variant="primary")
                export_btn = gr.Button("📥 Export Knowledge (JSON)", variant="secondary")
            
            analysis_output = gr.Textbox(
                label="Analysis Results",
                interactive=False,
                lines=20
            )

            stats_btn.click(get_session_stats, outputs=analysis_output)
            eval_btn.click(run_sample_evaluation, outputs=analysis_output)
            export_btn.click(export_knowledge, outputs=analysis_output)

print("✅ UI Fixed: Multiple files can be selected and ingested automatically.")

✅ UI Fixed: Multiple files can be selected and ingested automatically.


In [93]:
# --- 4.6: Launch Server ------------------------------------------------
gr.close_all()

try:
    app.close()
except:
    pass

app.launch(
    debug=True,
    share=False,
    server_name="localhost",
    css=custom_css,
    theme=gr.themes.Soft(primary_hue='emerald')
)

Closing server running on port: 7860
* Running on local URL:  http://localhost:7860
* To create a public link, set `share=True` in `launch()`.


  Loaded 1 page(s) from "contract_test.pdf"

✅ Ingested 2 chunks from "contract_test.pdf"

  Loaded 1 page(s) from "contract_test.pdf"

✅ Ingested 2 chunks from "contract_test.pdf"

  Loaded 1 page(s) from "مثال عقد.docx"

✅ Ingested 2 chunks from "مثال عقد.docx"

Keyboard interruption in main thread... closing server.
